In [1]:
import numpy as np
import imutils
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
# 路徑
path = './images/test_01.png'
# 正確答案
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

In [3]:
def cv_show(img, name='img'):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()  

def order_points(pts):
    rect = np.zeros((4, 2), dtype='float32')
    # 0123 分別是左上 右上 右下 左下
    # 計算左上 右下
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    # 計算右上 左下
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def four_point_transform(image, pts):
    # 獲取輸入座標
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    
    # 計算輸入的w & h
    # 計算上下邊寬度, 取最大值(因為有近似所以可能有差)
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # 計算左右邊高度, 取最大值(因為有近似所以可能有差)
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # 變換後對應座標位置
    # 目標點
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # 計算變換矩陣
    # 投影變換
    # 變換矩陣 M
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    
    return warped
# 排序方式
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][i], reverse=reverse))
    return cnts, boundingBoxes

In [4]:
image = cv2.imread(path)
# 答題卡
cv_show(image)

In [5]:
# 前處理
contours_img = image.copy()
# 灰階
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv_show(gray)
# 高斯模糊降躁
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
cv_show(blurred)
# 邊緣檢測
edged = cv2.Canny(blurred, 75, 200)
cv_show(edged)

In [6]:
# 輪廓檢測
cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(contours_img, cnts, -1, (0, 0, 255), 3) # 繪製圖像, 輪廓, 輪廓索引, 顏色模式(B, G, R), 線條厚度
cv_show(contours_img)
doc_cnt = None

In [7]:
# 檢查及排序
if len(cnts) > 0:
    # 排序
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True) # 依面積排序  
    for c in cnts:
        # 近似
        # 對contour 做多邊形逼近, 用周長百分比, 越小越像
        peri = cv2.arcLength(c, True) # 周長
        approx = cv2.approxPolyDP(c, 0.02 * peri, True) 
        # 準備透視變換
        if len(approx) == 4:
            doc_cnt = approx
            break   
    

In [8]:
# 透視變換
warped = four_point_transform(gray, doc_cnt.reshape(4, 2))
cv_show(warped)
# Otsu's 閾值處理
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1] 
cv_show(thresh)


In [9]:
thresh_Contours = thresh.copy()
# 找到每一個圓圈輪廓
cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(thresh_Contours, cnts, -1, (0, 0, 255), 3) 
cv_show(thresh_Contours)

In [10]:
question_cnts = []
for c in cnts:
    # 計算比例和大小
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)

    # 指定標準
    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        question_cnts.append(c)



In [11]:
# 按照從上到下排序
question_cnts = sort_contours(question_cnts, method="top-to-bottom")[0]
correct = 0
# 每排有五個選項
for (q, i) in enumerate(np.arange(0, len(question_cnts), 5)):
    # 排序
    cnts = sort_contours(question_cnts[i: i+5])[0]
    bubbled = None

    for (j, c) in enumerate(cnts):
        # 使用 mask來判斷結果
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1) #-1表示填充
        
        # 通過計算 != 0的數量來計算是否選擇該答案
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        cv_show(mask)
        total = cv2.countNonZero(mask)
#         print(f'{q, j}: {total}')

        # 通過閾值判斷
        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)

    # 對比正確答案
    color = (0, 0, 255)
    k = ANSWER_KEY[q]
    # 判斷正確
    if k == bubbled[1]:
        color = (0, 255, 0)
        correct += 1
    # 繪圖
    cv2.drawContours(warped, [cnts[k]], -1, color, 3)
    
score = (correct / 5.0) * 100
print(f"[INFO] score: {score}%")
cv2.putText(warped, f"{score}%", (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

cv_show(image)
cv_show(warped)

[INFO] score: 80.0%
